In [1]:
import os
import numpy as np
from io_f import read_data_file
from main import extract_ibeacon_rssi, compute_step_positions, split_ts_seq
from visualize_f import visualize_heatmap, save_figure_to_html
import json
from pathlib import Path
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
from PIL import Image
from collections import defaultdict

pio.kaleido.scope.default_format = "jpg"

def process_floor(site, floor, data_dir, output_dir):
    path_data_files = os.path.join(data_dir, site, floor, 'path_data_files')
    floor_path = os.path.join(data_dir, site, floor)
    floor_info_path = os.path.join(floor_path, 'floor_info.json')
    geojson_path = os.path.join(floor_path, 'geojson_map.json')
    floor_image_path = os.path.join(floor_path, 'floor_image.png')
    print('path_data_files:', path_data_files)
    print('floor_info_path:', floor_info_path)
    
    # Load floor information
    with open(floor_info_path) as f:
        floor_info = json.load(f)
    width_meter = floor_info["map_info"]["width"]
    height_meter = floor_info["map_info"]["height"]

    # Load GeoJSON data
    with open(geojson_path) as f:
        geojson_data = json.load(f)

    # Load floor plan image
    floor_plan = Image.open(floor_image_path)

    # Dictionary to store consolidated data for all iBeacons
    consolidated_ibeacon_data = defaultdict(list)

    # Process all path data files in the floor
    for filename in os.listdir(path_data_files):
        print('filename:', filename)
        if filename.endswith('.txt'):
            data_filename = os.path.join(floor_path, 'path_data_files', filename)
            path_data = read_data_file(data_filename)

            # Extract iBeacon RSSI data
            mwi_datas = {}
            acce_datas = path_data.acce
            ahrs_datas = path_data.ahrs
            ibeacon_datas = path_data.ibeacon
            posi_datas = path_data.waypoint

            step_positions = compute_step_positions(acce_datas, ahrs_datas, posi_datas)

            if ibeacon_datas.size != 0:
                sep_tss = np.unique(ibeacon_datas[:, 0].astype(float))
                ibeacon_datas_list = split_ts_seq(ibeacon_datas, sep_tss)
                for ibeacon_ds in ibeacon_datas_list:
                    diff = np.abs(step_positions[:, 0] - float(ibeacon_ds[0, 0]))
                    index = np.argmin(diff)
                    target_xy_key = tuple(step_positions[index, 1:3])
                    if target_xy_key in mwi_datas:
                        mwi_datas[target_xy_key]['ibeacon'] = np.append(mwi_datas[target_xy_key]['ibeacon'], ibeacon_ds, axis=0)
                    else:
                        mwi_datas[target_xy_key] = {
                            'magnetic': np.zeros((0, 4)),
                            'wifi': np.zeros((0, 5)),
                            'ibeacon': ibeacon_ds
                        }

            ibeacon_rssi = extract_ibeacon_rssi(mwi_datas)

            # Add data to consolidated dictionary
            for ibeacon, rssi_data in ibeacon_rssi.items():
                for pos, value in rssi_data.items():
                    consolidated_ibeacon_data[ibeacon].append((pos, value[0]))

            # Visualize iBeacon RSSI for all available iBeacons
            floor_output_dir = os.path.join(output_dir, site, floor, 'ibeacon_images')
            Path(floor_output_dir).mkdir(parents=True, exist_ok=True)

            # Check if there are any iBeacons to visualize
            if not ibeacon_rssi:
                print(f"No iBeacon data found in the dataset for {site}/{floor}/{filename}")
                fig = go.Figure()
                fig.add_annotation(
                    text="No iBeacon data available",
                    xref="paper", yref="paper",
                    x=0.5, y=0.5, showarrow=False
                )
                fig.write_image(f'{floor_output_dir}/no_ibeacon_data_{filename[:-4]}.jpg')
            else:
                # Create a plot for each iBeacon
                for ibeacon, rssi_data in ibeacon_rssi.items():
                    fig = go.Figure()

                    # Add floor plan image
                    fig.add_layout_image(
                        dict(
                            source=floor_plan,
                            xref="x",
                            yref="y",
                            x=0,
                            y=height_meter,
                            sizex=width_meter,
                            sizey=height_meter,
                            sizing="stretch",
                            opacity=0.5,
                            layer="below"
                        )
                    )

                    heat_positions = np.array(list(rssi_data.keys()))
                    heat_values = np.array(list(rssi_data.values()))[:, 0]
                    
                    scatter = go.Scatter(
                        x=heat_positions[:, 0],
                        y=heat_positions[:, 1],
                        mode='markers',
                        marker=dict(
                            size=10,
                            color=heat_values,
                            colorscale='Viridis',
                            showscale=True,
                            colorbar=dict(title='RSSI (dBm)', y=0.5, len=0.75, yanchor='middle'),
                        ),
                        text=[f'RSSI: {v:.2f} dBm' for v in heat_values],
                        hoverinfo='text',
                        name=ibeacon
                    )
                    
                    fig.add_trace(scatter)

                    # Update layout
                    fig.update_layout(
                        title=f'iBeacon RSSI Heatmap: {ibeacon} - {site}/{floor}/{filename[:-4]}',
                        autosize=True,
                        width=1200,
                        height=1200 * (height_meter / width_meter),  # Maintain aspect ratio
                        showlegend=False,
                    )

                    # Update axes
                    fig.update_xaxes(title_text="X Position (m)", range=[0, width_meter])
                    fig.update_yaxes(title_text="Y Position (m)", range=[0, height_meter], scaleanchor="x", scaleratio=1)

                    # Save the figure as a JPG file
                    jpg_filename = f'{floor_output_dir}/ibeacon_rssi_{ibeacon.replace(":", "_")}_{filename[:-4]}.jpg'
                    print('jpg_filename:', jpg_filename)
                    fig.write_image(jpg_filename, scale=2)  # scale=2 for higher resolution
                    print(f"Visualization for iBeacon {ibeacon} saved to {jpg_filename}")

    # Create consolidated visualization for the floor
    fig = go.Figure()

    # Add floor plan image
    fig.add_layout_image(
        dict(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="stretch",
            opacity=0.5,
            layer="below"
        )
    )

    # Collect all RSSI values to determine the global range
    all_rssi_values = []
    for data_list in consolidated_ibeacon_data.values():
        all_rssi_values.extend([value for _, value in data_list])

    min_rssi = min(all_rssi_values)
    max_rssi = max(all_rssi_values)

    # Add data for each iBeacon
    for i, (ibeacon, data_list) in enumerate(consolidated_ibeacon_data.items()):
        positions, values = zip(*data_list)
        x, y = zip(*positions)
        
        scatter = go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker=dict(
                size=10,
                color=values,
                colorscale='Viridis',
                cmin=min_rssi,
                cmax=max_rssi,
                showscale=i == 0,  # Only show color bar for the first trace
                colorbar=dict(title='RSSI (dBm)', y=0.5, len=0.75, yanchor='middle') if i == 0 else None,
            ),
            text=[f'iBeacon: {ibeacon}, RSSI: {v:.2f} dBm' for v in values],
            hoverinfo='text',
            name=ibeacon,
            legendgroup=f'group{i}',
            showlegend=True
        )
        
        fig.add_trace(scatter)

    fig.update_layout(
        autosize=True,
        width=1200,
        height=1200 * (height_meter / width_meter),
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1,
            itemsizing='constant'
        ),
        margin=dict(b=50),  # Add bottom margin
        annotations=[
        dict(
            x=0.5,
            y=0.1,  # Move the title slightly below the plot area
            #text=f'Consolidated iBeacon RSSI Heatmap - {site}/{floor}',
            text=f'Consolidated iBeacon_{site}--{floor}',
            showarrow=False,
            xref='paper',
            yref='paper',
            xanchor='center',
            yanchor='top',  # Change to 'top' since we moved it below
            font=dict(size=14, weight='bold')  # Make the font bold
        )
        ]
    )

    # Update axes
    fig.update_xaxes(title_text="X Position (m)", range=[0, width_meter])
    fig.update_yaxes(title_text="Y Position (m)", range=[0, height_meter], scaleanchor="x", scaleratio=1)

    # Save the consolidated figure as a JPG file
    ibeacon_consolidated_floor_dir=os.path.join(output_dir, site)

    # consolidated_jpg_filename = f'{floor_output_dir}/consolidated_ibeacon_{site}--{floor}.jpg'
    consolidated_jpg_filename = f'{ibeacon_consolidated_floor_dir}/consolidated_ibeacon_{site}--{floor}.jpg'
    fig.write_image(consolidated_jpg_filename, scale=2)  # scale=2 for higher resolution
    print(f"Consolidated visualization for {site}/{floor} saved to {consolidated_jpg_filename}")


def main():
    data_dir = './data'
    output_dir = './output'

    for site in os.listdir(data_dir):
        site_path = os.path.join(data_dir, site)
        if os.path.isdir(site_path):
            for floor in os.listdir(site_path):
                floor_path = os.path.join(site_path, floor)
                if os.path.isdir(floor_path):
                    print(f"Processing {site}/{floor}")
                    process_floor(site, floor, data_dir, output_dir)

    print("All visualizations completed.")

if __name__ == "__main__":
    main()




Processing site2/F4
path_data_files: ./data/site2/F4/path_data_files
floor_info_path: ./data/site2/F4/floor_info.json
filename: 5dd52602d48f840006f14a4f.txt
jpg_filename: ./output/site2/F4/ibeacon_images/ibeacon_rssi_FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10073_61418_5dd52602d48f840006f14a4f.jpg
Visualization for iBeacon FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10073_61418 saved to ./output/site2/F4/ibeacon_images/ibeacon_rssi_FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10073_61418_5dd52602d48f840006f14a4f.jpg
filename: 5dd3ac5444333f00067aa662.txt
No iBeacon data found in the dataset for site2/F4/5dd3ac5444333f00067aa662.txt
filename: 5dd3bbd627889b0006b76fcb.txt
No iBeacon data found in the dataset for site2/F4/5dd3bbd627889b0006b76fcb.txt
filename: 5dd3a32544333f00067aa5ab.txt
jpg_filename: ./output/site2/F4/ibeacon_images/ibeacon_rssi_FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10065_26049_5dd3a32544333f00067aa5ab.jpg
Visualization for iBeacon FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10065_26049 saved to 